In [ ]:
import sys
import os
import math
import time
import numpy as np
from PIL import Image
import cv2
import pynq
import scipy.misc
from IPython.display import display

In [ ]:
overlay = pynq.Overlay('ConvNet.bit')
dma = overlay.axi_dma_0
xlnk = pynq.Xlnk()
nn_ctrl = overlay.convnet_0

In [ ]:
img = cv2.imread("image_1042.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (160, 160))
display(scipy.misc.toimage(img ))

In [ ]:
in_buffer = xlnk.cma_array(shape=(160, 160, 3), dtype=np.uint8, cacheable=1)    
out_buffer = xlnk.cma_array(shape=(16), dtype=np.int32, cacheable=1) 

In [ ]:
def net(in_buffer, out_buffer):
    nn_ctrl.write(0x0, 0) # Reset
    nn_ctrl.write(0x10, 1)#reps
    nn_ctrl.write(0x0, 1) # Deassert reset
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.transfer(in_buffer)      
    dma.sendchannel.wait()
    dma.recvchannel.wait()

In [ ]:
start = time.time()
in_buffer[:] = img
net(in_buffer, out_buffer)
end = time.time()
result = out_buffer
result = result.tolist()
max_index = result.index(max(result))
print(result)
print(max_index)

In [ ]:
total_time = end - start
print("Total time:", total_time, "seconds")
print('fps: {} .'.format(1 / total_time))